In [3]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle
from sklearn.cluster import KMeans
from scipy.spatial import distance
import warnings
warnings.filterwarnings('ignore')

In [11]:
dataset_name = 'C:/Users/Vishal/Downloads/datasets/datasets/spambase.csv'
data = pd.read_csv(dataset_name)
THRESHOLD=0.95
Q=5
BETA=0.1
pool={}
# data = data.sample(frac=1).reset_index(drop=True)
# data = data.apply(pd.to_numeric).fillna(0)
# data = data.iloc[:, 1:]
data

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X49,X50,X51,X52,X53,X54,X55,X56,X57,Y
0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.000,0.000,0.0,0.000,0.000,0.000,1.214,4,17,0
1,0.05,0.30,0.40,0.0,0.10,0.05,0.0,0.05,0.10,0.00,...,0.000,0.036,0.0,0.054,0.118,0.000,2.379,96,583,1
2,0.00,0.76,0.00,0.0,0.00,0.00,0.0,0.00,0.57,0.19,...,0.000,0.099,0.0,0.232,0.066,0.928,20.432,213,1655,1
3,0.09,0.49,0.59,0.0,0.39,0.19,0.0,0.00,0.09,0.39,...,0.765,0.037,0.0,5.828,1.308,0.000,6.047,54,768,1
4,0.00,0.00,0.78,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.000,0.401,0.0,0.133,0.000,0.000,1.565,4,36,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.000,0.000,0.0,0.000,0.000,0.000,1.000,1,5,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.000,0.000,0.0,0.000,0.000,0.000,1.000,1,4,0
4598,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.000,0.000,0.0,0.000,0.000,0.000,1.500,9,30,0
4599,0.00,0.00,0.26,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.036,0.109,0.0,0.000,0.036,0.000,1.632,11,307,0


In [5]:
def create_classifier(data,Q,batch_split):
    full_batch=data.iloc[:,:-1]
    kmeans = KMeans(n_clusters=Q, random_state=0).fit(full_batch)
    labelled_batch=data.iloc[:batch_split,:]
    labelled_batch['pred']=kmeans.labels_[:batch_split]
    df=labelled_batch.iloc[:,-2:]
    a=df.groupby(['pred']).agg(lambda x:x.value_counts().index[0])
    mapping={}
    b=a.reset_index()
    for i in range(0,len(b['pred'])):
        mapping[b['pred'][i]]=b.iloc[:,-1][i] 
    return kmeans,mapping

In [6]:
def update_pool(labelled_batch,batch_no,full_batch):
    batch_simi={}
    for classifier_name in pool:
        similarity=find_accuracy(classifier_name,labelled_batch)
        batch_simi[classifier_name]=similarity
        
    max_similarity_key=max(batch_simi, key=batch_simi.get)
    max_similarity = batch_simi[max_similarity_key]
    model,mapping=create_classifier(full_batch,Q,batch_split)
    if(max_similarity>THRESHOLD or len(pool)>=max_c):
        model_file=open(max_similarity_key,'wb')
        map_file=open(max_similarity_key.split(".")[0],'wb')
        pickle.dump(model,model_file)
        pickle.dump(mapping,map_file)
        pool[max_similarity_key]=1
    else:
        model,mapping=create_classifier(full_batch,Q,batch_split)
        new_batch="Classifier"+str(batch_no)+".sav"
        new_map="Classifier"+str(batch_no)
        model_file=open(new_batch,'wb')
        map_file=open(new_map,'wb')
        pickle.dump(model,model_file)
        pickle.dump(mapping,map_file)
        pool[new_batch]=1
    return pool

In [7]:
def find_accuracy(classifier_name,labelled_batch):
    s=0
    model_file=open(classifier_name,'rb')
    map_file=open(classifier_name.split(".")[0],'rb')
    model=pickle.load(model_file)
    mapping=pickle.load(map_file)
    for i in range(len(labelled_batch)):
        current_row=labelled_batch.iloc[i:i+1,:-1]
        current_label=labelled_batch.iloc[i:i+1,-1]
        pred=model.predict(current_row)[0]
        if(pred not in mapping.keys()):
                continue
        final_pred=mapping[pred]
        if(final_pred==current_label[i]):
            s+=1
    return((s)/len(labelled_batch))

In [8]:
def find_distance(a, b):
    return distance.euclidean(a, b)

In [9]:
def return_nearest_index(current_row, centroids,mapping):
    dist={}
    for x in mapping.keys():
        dist[x]=np.linalg.norm(current_row - centroids[x])
    #if (len(sorted(dist.items(), key = lambda kv:(kv[1], kv[0])))==1):
        #return sorted(dist.items(), key = lambda kv:(kv[1], kv[0]))[0]
    return sorted(dist.items(), key = lambda kv:(kv[1], kv[0]))[0]

In [ ]:
pool={}
batch_no=1
stream_size=int(100)
skip=0
batch_split = int(0.2 * stream_size)
max_c=10
acc=[]
#for j in range(0,4):
for j in range(0,int(len(data)/stream_size)):
    print("Batch",batch_no)
    batch = pd.read_csv(dataset_name, nrows = stream_size, skiprows=skip)
    
    if(j==0):
        model_name="Classifier"+str(batch_no)+".sav"#Initial classifier
        map_name="Classifier"+str(batch_no)
        model,mapping=create_classifier(batch,Q,batch_split)
        pool[model_name]=1
        model_file=open(model_name,'wb')
        map_file=open(map_name,'wb')
        pickle.dump(model,model_file)
        pickle.dump(mapping,map_file)
    else:
        acc=[]
        s=0
        labelled_batch=batch.iloc[:batch_split,:]
        
        for i in range(len(labelled_batch)):
            current_row=labelled_batch.iloc[i:i+1,:-1]
            current_label=labelled_batch.iloc[i:i+1,-1]
            Keymax = max(pool, key=pool.get)
            model_file=open(Keymax,'rb')
            map_file=open(Keymax.split(".")[0],'rb')
            model=pickle.load(model_file)
            mapping=pickle.load(map_file)
            pred=model.predict(current_row)[0]
            #print("centroids: " + str(model.cluster_centers_))
            for classifier_name in pool:            
                model_file=open(classifier_name,'rb')
                map_file=open(classifier_name.split(".")[0],'rb')
                model=pickle.load(model_file)
                mapping=pickle.load(map_file)
                #pred=model.predict(current_row)[0]
                #if(pred not in mapping.keys()):
                    #centroids = model.cluster_centers_
                    #pred = return_nearest_index(current_row, centroids,mapping)[0]
                centroids = model.cluster_centers_
                pred = return_nearest_index(current_row, centroids,mapping)[0]
                final_pred=mapping[pred]
                if(final_pred!=current_label[i]):
                    pool[classifier_name]=pool[classifier_name]*(BETA**1)#Reduce weight otherwise
        unlabelled_batch=batch.iloc[batch_split:,:]
        for i in range(len(unlabelled_batch)):
            current_row=unlabelled_batch.iloc[i:i+1,:-1]
            current_label=unlabelled_batch.iloc[i:i+1,-1]
            Keymax = max(pool, key=pool.get)
            model_file=open(Keymax,'rb')
            map_file=open(Keymax.split(".")[0],'rb')
            model=pickle.load(model_file)
            mapping=pickle.load(map_file)
            #pred=model.predict(current_row)[0]
            #if(pred not in mapping.keys()):
                #find other nearest cluster
                #centroids = model.cluster_centers_
                #pred = return_nearest_index(current_row, centroids,mapping)[0]
            centroids = model.cluster_centers_
            pred = return_nearest_index(current_row, centroids,mapping)[0]
            final_pred=mapping[pred]
            if(final_pred==current_label[i+batch_split]):
                s+=1
        acc.append([batch_no,s/len(unlabelled_batch)])
        pool=update_pool(labelled_batch,batch_no,batch)#Update pool
    print(acc)
    skip += stream_size
    batch_no += 1
    

Batch 1
[]
Batch 2
[[2, 0.675]]
Batch 3
[[3, 0.5375]]
Batch 4
[[4, 0.725]]
Batch 5
[[5, 0.6125]]
Batch 6
[[6, 0.7375]]
Batch 7
[[7, 0.6625]]
Batch 8
[[8, 0.675]]
Batch 9
[[9, 0.75]]
Batch 10
[[10, 0.625]]
Batch 11
[[11, 0.675]]
Batch 12
[[12, 0.6875]]
Batch 13
[[13, 0.6]]
Batch 14
[[14, 0.7125]]
Batch 15
[[15, 0.6875]]
Batch 16
[[16, 0.7375]]
Batch 17
[[17, 0.7875]]
Batch 18
[[18, 0.6]]
Batch 19
[[19, 0.725]]
Batch 20
[[20, 0.5875]]
Batch 21
[[21, 0.3625]]
Batch 22
[[22, 0.6375]]
Batch 23
[[23, 0.5125]]
Batch 24
